In [1]:
import pandas as pd
import numpy as np
import os
import re
import datetime as dt
import calendar

import plotly.graph_objs as go
import plotly.offline as pyo
import plotly.io as pio
pyo.init_notebook_mode(connected=True)
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.tsa.seasonal import seasonal_decompose

from scipy.optimize import minimize
import scipy.stats as scs

from sklearn.metrics import r2_score, median_absolute_error,mean_squared_error, mean_squared_log_error, mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit

In [2]:
if not os.path.exists('graphiques'):
    os.mkdir('graphiques')

In [3]:
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 500)

In [4]:
!ls './data/eCO2'

eCO2mix_RTE_Annuel-Definitif_2012.csv  eCO2mix_RTE_Annuel-Definitif_2015.csv
eCO2mix_RTE_Annuel-Definitif_2013.csv  eCO2mix_RTE_Annuel-Definitif_2016.csv
eCO2mix_RTE_Annuel-Definitif_2014.csv  eCO2mix_RTE_Annuel-Definitif_2017.csv


### <font color='blue'>Merging dataframe from files<font>

In [ ]:
df_raw = pd.DataFrame()
tot_length=0
for file in os.listdir('data/eCO2'):
    df_to_merge=pd.read_csv('./data/eCO2/{}'.format(file), low_memory=False)#set low_memory=False to ignore mixed type in columns (handled later)
    length = len(df_to_merge)
    print(length)
    tot_length +=length
    df_raw = df_raw.append(df_to_merge)
print(tot_length)
df_raw=df_raw.reset_index(drop=True)
display(df_raw.head(2))

35136
35040
35136
35040
35040
35040
210432


,Perimetre,Nature,Date,Heures,Consommation,Prevision J-1,Prevision J,Fioul,Charbon,Gaz,Nucleaire,Eolien,Solaire,Hydraulique,Pompage,Bioenergies,Ech. physiques,Taux de Co2,Ech. comm. Angleterre,Ech. comm. Espagne,Ech. comm. Italie,Ech. comm. Suisse,Ech. comm. Allemagne-Belgique,Fioul - TAC,Fioul - Cogen.,Fioul - Autres,Gaz - TAC,Gaz - Cogen.,Gaz - CCG,Gaz - Autres,Hydraulique - Fil de l?eau + eclusee,Hydraulique - Lacs,Hydraulique - STEP turbinage,Bioenergies - Dechets,Bioenergies - Biomasse,Bioenergies - Biogaz
0,France,Donnees definitives,2012-01-01,00:00,58315.0,58200,58200,492.0,25.0,3816.0,52697.0,3588.0,0.0,7922.0,-1139.0,719.0,-9806.0,33.0,-1750,-1200,-862,-2625,-2940,ND,ND,ND,ND,ND,ND,ND,ND,ND,ND,ND,ND,ND
1,France,Donnees definitives,2012-01-01,00:15,NaN,57700,57550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### <font color='blue'>Files (each year) don't have the same number of record of electricity consumption, and some NaN data appear<font>

In [ ]:
df_eCO2=df_raw.iloc[:,0:7].copy()
display(df_eCO2.head(2))

,Perimetre,Nature,Date,Heures,Consommation,Prevision J-1,Prevision J
0,France,Donnees definitives,2012-01-01,00:00,58315.0,58200,58200
1,France,Donnees definitives,2012-01-01,00:15,NaN,57700,57550


In [ ]:
df_eCO2['Date']=df_eCO2['Date'].astype('datetime64[ns]')
df_eCO2=df_eCO2.sort_values(by=['Date','Heures'],ascending=True)

In [ ]:
#column full datetime (sec)
df_eCO2['Datetime'] = pd.to_datetime(df_eCO2['Date'].apply(str)+' '+df_eCO2['Heures'])
df_eCO2['Datetime']=df_eCO2['Datetime'].astype('datetime64[ns]')
df_eCO2.head(2)

In [ ]:
data=df_eCO2.dropna().copy()
print(len(data))

x=data['Datetime']
y1=data['Consommation']


trace1 = go.Scatter(
    x=x,
    y=y1,
    name = "Consommation",
    line = dict(color = '#17BECF'),
    opacity = 0.8)

layout = dict(
    title=dict(text="<b>Consommation électrique<b>",
                   font=dict(family='Linux Biolinum O',size=18,color='black')),
    xaxis=dict(title='Datum',
        rangeselector=dict(
            buttons=list([
                dict(count=7,
                     label='7d',
                     step='day',
                     stepmode='backward'),
                dict(count=3,
                     label='3m',
                     step='month',
                     stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(
            visible = True
        ),
        type='date'),
    yaxis=dict(title='KW/H')
)

data = [trace1]

fig = go.Figure(data=data, layout=layout)

pyo.iplot(fig)

La  variance  ne  semble  pas  toujours  constante autour  de  la  moyenne  saisonnière à  court  terme  mais  elle reste  relativement  stable  sur  des  périodes  plus  longues. 

### Comsumption have a year, week and a daily cycle. Called seasonalities

### Also, summer afternoon and winter afternoon are very differents

In [ ]:
def add_datepart(df, fldname, drop=False, time=False):
    
    import re
    
    "Add columns relevant to a date in the column `fldname` of `df`."
    
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]atetime$', '', fldname)
    #attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            #'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    attr=['Year', 'Month', 'Week', 'Day']
    if time: attr = attr + ['Hour', 'Minute', 'Second', 'Dayofweek']
    
    '''à tester:
    -sans la ligne suivante pour enlever cette colonne,
    -et de façon isoler pour la corriger'''
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)

add_datepart(df_eCO2,'Datetime',time=True)
df_eCO2.head(2)

### <font color='blue'>NaN Data?<font>

In [ ]:
len(df_eCO2)

In [ ]:
pd.DataFrame(df_eCO2.isna().sum()).T

In [ ]:
#NaN time minutes localisation
for quarter in df_eCO2['Minute'].unique():
    df=df_eCO2[df_eCO2['Minute']==quarter]
    df=pd.DataFrame(df.isna().sum()).T
    count=int(df['Consommation'][0])
    print(quarter,':',count)

In [ ]:
df_eCO2.loc[df_eCO2['Consommation'].isnull()].head(2)

### <font color='red'>Every xx:15 and xx:45 records in 'Consomation' are NaN<font>  
### <font color='green'>For now, i consider this data are not necessary but to be removed to ligth data. We could eventually estimate them by refering to previsions and consommation.<font>  

In [ ]:
df_eCO2=df_eCO2.dropna()

### <font color='red'>In cell n°5 (merging), noted that 2 Dataframes are longer than other. More preciasily:<font>

In [ ]:
#localisation
for year in df_eCO2['Year'].unique():
    df=df_eCO2[df_eCO2['Year']==year]
    l=len(df)
    print(year,':',l)

In [ ]:
#days 17520 
(17568-17520)/2/24#because data have 30 mins frequency.

##### <font color='green'>one day is recorded in 2012 and 2016, but not in 2013,2015,2016,2017
#### <font color='red'>Wich days are missing?
##### <font color='green'>Pbly in february<font>

In [ ]:
#localisation
for year in df_eCO2['Year'].unique():
    df=df_eCO2[(df_eCO2['Year']==year)&(df_eCO2['Month']==2.0)]
    l=int(len(df))/48
    print(year,':',l)

In [ ]:
for year in df_eCO2['Year'].unique():
    df = df_eCO2[(df_eCO2['Year']==year)&(df_eCO2['Month']==2.0)&
                (df_eCO2['Day']==29.0)]
    display(df.head(1))

### <font color='green'>That's it: years bisextile and not<font>

### <font color='red'>The fact that  2012 and 2016 has one day more can be a source of problems. A specialy for prediction and machine learning<font>

In [ ]:
data=df_eCO2.copy()
N = df_eCO2['Dayofweek'].unique()     # boxes (dayofweek)
c = ['hsl('+str(h)+',50%'+',50%)' for h in np.linspace(0, 360, len(N))]#color generator
# Each box is represented by a dict that contains the data, the type, and the colour. 
data = [{
    'y': data['Consommation'][data['Dayofweek']==m], 
    'name':str(calendar.day_name[m]),
    'type':'box',
    'marker':{'color': c[m-1]}
    } for m in np.arange(0,len(N),1)]

layout = go.Layout(title=dict(text="<b>Consommation hebdomadaire<b>",
                   font=dict(family='Linux Biolinum O',size=18,color='black')),
                   xaxis=dict(title='Dayofweek',showgrid=False,zeroline=False, tickangle=-45,showticklabels=True),
                   yaxis=dict(title='Consommation',gridcolor='white', zeroline=False),
                   paper_bgcolor='rgb(233,233,233)',
                   plot_bgcolor='rgb(233,233,233)',
                  )

fig = go.Figure(data=data, layout=layout)

pyo.iplot(fig)


In [ ]:
data=df_eCO2.copy()
N = df_eCO2['Month'].unique()     # boxes (months)
c = ['hsl('+str(h)+',50%'+',50%)' for h in np.linspace(0, 360, len(N))]#color generator
# Each box is represented by a dict that contains the data, the type, and the colour. 
data = [{
    'y': data['Consommation'][data['Month']==m], 
    'name':str(calendar.month_name[m]),
    'type':'box',
    'marker':{'color': c[m-1]}
    } for m in np.arange(0,len(N)+1,1)]

layout = go.Layout(title=dict(text="<b>Consommation mensuelle<b>",
                   font=dict(family='Linux Biolinum O',size=18,color='black')),
                   xaxis=dict(title='Months',showgrid=False,zeroline=False, tickangle=-45,showticklabels=True),
                   yaxis=dict(title='Consommation',gridcolor='white', zeroline=False),
                   paper_bgcolor='rgb(233,233,233)',
                   plot_bgcolor='rgb(233,233,233)',
                  )

fig = go.Figure(data=data, layout=layout)

pyo.iplot(fig)


In [ ]:
data=df_eCO2.copy()
N = df_eCO2['Year'].unique()     # boxes (years)
c = ['hsl('+str(h)+',50%'+',50%)' for h in np.linspace(0, 360, len(N))]#color generator
# Each box is represented by a dict that contains the data, the type, and the colour. 
data = [{
    'y': data['Consommation'][data['Year']==m], 
    'name':str(m),
    'type':'box',
    'marker':{'color': c[m-2012-1]}
    } for m in list(N)]

layout = go.Layout(title=dict(text="<b>Consommation annuelle<b>",
                   font=dict(family='Linux Biolinum O',size=18,color='black')),
                   xaxis=dict(title='Year',showgrid=False,zeroline=False, tickangle=-45,showticklabels=True),
                   yaxis=dict(title='Consommation',gridcolor='white', zeroline=False),
                   paper_bgcolor='rgb(233,233,233)',
                   plot_bgcolor='rgb(233,233,233)',
                  )

fig = go.Figure(data=data, layout=layout)

pyo.iplot(fig)


In [ ]:
data=df_eCO2.copy()
years = list(data['Year'].unique())

x=data['Datetime']
y1=data['Consommation']
y2=data['Prevision J-1']
y3=data['Prevision J']


trace1 = go.Line(x=x,
                  y=y1,
                  opacity = 1,
                  name='Consommation',
                  marker=dict(color='green'))

trace2 = go.Line(x=x,
                  y=y2,
                  opacity = 0.6,
                  name='Prevision J-1',
                  marker=dict(color='blue'))

trace3 = go.Line(x=x,
                  y=y3,
                  opacity = 0.6,
                  name='Prevision J',
                  marker=dict(color='red'))

data = [trace1,trace2,trace3]

layout = dict(
    title=dict(text="<b>Consommation et prédictions<b>",
               font=dict(family='Linux Biolinum O',size=18,color='black')),
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label='1m',
                     step='month',
                     stepmode='backward'),
                dict(count=6,
                     label='6m',
                     step='month',
                     stepmode='backward'),
                dict(count=1,
                    label='1y',
                    step='year',
                    stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(
            visible = True
        ),
        type='date'
    )
)

fig = dict(data=data, layout=layout)
pyo.iplot(fig)

In [ ]:
data=df_eCO2.copy()

print(len(data))

x=data['Datetime']
y1=data['Consommation']-data['Prevision J-1']
y2=data['Consommation']-data['Prevision J']


trace1 = go.Line(x=x,
                  y=y1,
                  name='Consommation-prévision J-1',
                  marker=dict(color='red'))

trace2 = go.Line(x=x,
                  y=y2,
                  name='Consommation-prévision J',
                  marker=dict(color='blue'))

data = [trace1,trace2]

layout = go.Layout(title=dict(text="<b><b>",
                              font=dict(family='Linux Biolinum O',size=18,color='black')),
                   xaxis=dict(title='Datum'),
                   yaxis=dict(title=''))

fig = go.Figure(data=data, layout=layout)

pyo.iplot(fig)


In [ ]:
print('Prevision J',df_eCO2['Month'][df_eCO2['Prevision J']==0].unique())
print('Prevision J',df_eCO2['Year'][df_eCO2['Prevision J']==0].unique())
print('Prevision J-1',df_eCO2['Month'][df_eCO2['Prevision J-1']==0].unique())
print('Prevision J-1',df_eCO2['Year'][df_eCO2['Prevision J-1']==0].unique())

In [ ]:
df_eCO2['Prevision J'][df_eCO2['Prevision J']==0].count()

### <font color='red'>94 forecasts ('Prevision J') for the month of March 2015 are inconsistent. It is opportune, considering the prediction purpose, to correct this by referring to the predictions of March of other years. <font>

In [ ]:
df_eCO2.head(2)

In [ ]:
#isolate datetime if predicition is <0 then list day and month
#and return mean prediction from other years, at the same time
list_mean=[]
list_datetime_no_prediction=list(df_eCO2['Datetime'][df_eCO2['Prevision J']==0])
#extract day month and time
for dt in list_datetime_no_prediction:
    day=int(df_eCO2.Day[df_eCO2['Datetime']==dt].values)
    month=int(df_eCO2.Month[df_eCO2['Datetime']==dt].values)
    tim=str(df_eCO2.Heures[df_eCO2['Datetime']==dt].values)
    tim=tim[2:7]
    i = int(df_eCO2.index[df_eCO2['Datetime']==dt].values)
    mean = df_eCO2['Prevision J'][(df_eCO2['Year'].isin(['2012','2013','2014','2016','2017']))&
                 (df_eCO2['Day']==day)&
                 (df_eCO2['Month']==month)&
                 (df_eCO2['Heures']==tim)
                  ].mean()
    df_eCO2.at[i,'Prevision J']=mean

In [ ]:
df_eCO2['Prevision J'][df_eCO2['Prevision J']==0].count()

In [ ]:
from random import randint

In [ ]:

data=df_eCO2.dropna().reset_index(drop=True)
rec_day = len(df_eCO2)/len(df_eCO2['Date'].unique())
week_i = 7*rec_day#week interval
r = randint(a=week_i,b=len(data)-week_i)
years = list(data['Year'].unique())
data = data[['Date','Datetime','Heures','Consommation','Prevision J-1','Prevision J']].loc[r-week_i/2:r+week_i/2,:]

x=data['Datetime']
y1=data['Consommation']
y2=data['Prevision J-1']
y3=data['Prevision J']


trace1 = go.Line(x=x,
                  y=y1,
                  opacity = 1,
                  name='Consommation',
                  marker=dict(color='green'))

trace2 = go.Line(x=x,
                  y=y2,
                  opacity = 0.6,
                  name='Prevision J-1',
                  marker=dict(color='blue'))

trace3 = go.Line(x=x,
                  y=y3,
                  opacity = 0.6,
                  name='Prevision J',
                  marker=dict(color='red'))

data = [trace1,trace2,trace3]

layout = dict(
    title=dict(text="<b>Consommation et prédictions, échelles plus fines & période aléatoire<b>",
               font=dict(family='Linux Biolinum O',size=18,color='black')),
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label='1d',
                     step='day',
                     stepmode='backward'),
                dict(count=7,
                     label='7d',
                     step='day',
                     stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(
            visible = True
        ),
        type='date'
    )
)

fig = dict(data=data, layout=layout)
pyo.iplot(fig)

In [ ]:
### seasons
data=df_eCO2.dropna().reset_index(drop=True)
data=data.groupby(by=['Month','Day','Heures'], as_index=False).mean()
x=data['Heures'].unique()
summer=data['Consommation'][(data['Day']==15)&(data['Month']==7)]
spring=data['Consommation'][(data['Day']==15)&(data['Month']==4)]
fall=data['Consommation'][(data['Day']==30)&(data['Month']==10)]
winter=data['Consommation'][(data['Day']==1)&(data['Month']==1)]

trace1 = go.Line(x=x,
                  y=summer,
                  opacity = 1,
                  name='Ete 15-07',
                  marker=dict(color='red'))

trace2 = go.Line(x=x,
                  y=spring,
                  opacity = 0.6,
                  name='Printemps 15-04',
                  marker=dict(color='green'))

trace3 = go.Line(x=x,
                  y=fall,
                  opacity = 0.6,
                  name='Automne 30-10',
                  marker=dict(color='black'))

trace4 = go.Line(x=x,
                  y=winter,
                  opacity = 0.6,
                  name='Hiver 01-01',
                  marker=dict(color='blue'))

data = [trace1,trace2,trace3,trace4]

layout = go.Layout(title=dict(text="<b>Consommation sur 24h selon la saison<b>",
                              font=dict(family='Linux Biolinum O',size=18,color='black')),
                   xaxis=dict(title='Datum'),
                   yaxis=dict(title='KW/H'))

fig = dict(data=data, layout=layout)
pyo.iplot(fig)

### Daylike has some specificities (daymean and evening) regarding to the season but general rythmus is almost the same:  
Consommation raises arround 5 am until noon, then drops (more or less) during after-noon, reraise arround 17:30 and appears some specificities during the evening, finally drops until morning.

Week and season point of view

In [ ]:
### week seasons
data=df_eCO2.dropna().reset_index(drop=True)
data=data.groupby(by=['Month','Dayofweek','Heures'], as_index=False).mean()

x=[]
hours=df_eCO2['Heures'].unique()
daysweeks=[calendar.day_name[m] for m in np.arange(0,7,1)]
for d in daysweeks:
    for h in hours:
        dh=str(d+' '+h)
        x.append(dh)
        
summer=data['Consommation'][data['Month']==7]
spring=data['Consommation'][data['Month']==4]
fall=data['Consommation'][data['Month']==10]
winter=data['Consommation'][data['Month']==1]

trace1 = go.Line(x=x,
                  y=summer,
                  opacity = 1,
                  name='July',
                  marker=dict(color='red'))

trace2 = go.Line(x=x,
                  y=spring,
                  opacity = 0.6,
                  name='April',
                  marker=dict(color='green'))

trace3 = go.Line(x=x,
                  y=fall,
                  opacity = 0.8,
                  name='October',
                  marker=dict(color='black'))

trace4 = go.Line(x=x,
                  y=winter,
                  opacity = 0.6,
                  name='January',
                  marker=dict(color='blue'))

data = [trace1,trace2,trace3,trace4]

layout = go.Layout(title=dict(text="<b>Consommation sur une semaine selon la saison<b>",
                              font=dict(family='Linux Biolinum O',size=18,color='black')),
                   xaxis=dict(title='Datum',
                              type='-',
                              nticks = 7,
                              autorange=True),
                   yaxis=dict(title='KW/H'))

fig = dict(data=data, layout=layout)
pyo.iplot(fig)

### <font color='red'>Weekday and Week-end seems different whatever the season

df_eCO2=df_eCO2[df_eCO2['Heures']=='14:00']
df_eCO2=df_eCO2.reset_index(drop=True)

In [ ]:
df_eCO2=df_eCO2.drop(columns='Elapsed')
df_eCO2.head(2)

Datetime has been lost in the battel

# <font color='orange'>Heat correction of the consumption data<font>
#### DJU are localy and monthly recorded. So we'll focus on NICE
The heat fixing consists of removing the extr, whatever the harshness of the winter. By reducing the heating consumption to a reference climate, characterized by the DJU, it removes the variations due to the climatic severity. Kind of outlier removal.  
formula:  
Cconsumption = consumption x (DJU_reference/DJU_day)  
    with:  
    DJU_reference = 18  
          DJU_day = download  
          consumption = download
    

In [ ]:
df_raw=pd.read_csv('./data/dju_nice.csv')
display(df_raw.head(2))

### <font color='blue'>Preparing for merging<font>

In [ ]:
l_months=list(df_raw.columns.unique()[1:13])
l_years = list(df_raw['Annee'].unique())

In [ ]:
df_DJU=pd.DataFrame()
df_DJU['Year']=np.repeat(l_years,len(l_months))
df_DJU['Month_l']=np.tile(l_months,len(l_years))
df_DJU['Month']=np.tile(np.arange(1,13,1),len(l_years))
df_DJU.head(2)

In [ ]:
from itertools import product

l_djus=[]
for y,m in product(l_years,l_months):
    df = df_raw[df_raw['Annee']==y].copy()
    dju = float(df[[m]].values)
    l_djus.append(dju)

In [ ]:
df_DJU['dju']=l_djus

In [ ]:
df_DJU.head(2)

In [ ]:
data=df_DJU.copy()
data = data.groupby(by=['Year','Month_l'],as_index=False).mean()
data = data.sort_values(by=['Year','Month'])

liste_m_y=[]
for y in l_years:
    for m in l_months:
        d = str(y) + str(m)
        liste_m_y.append(d)

x=liste_m_y
y1=data['dju']
trace1 = go.Line(x=x,
                  y=y1,
                  name='DJU mensuel',
                  marker=dict(color='red'))

data = [trace1]

layout = go.Layout(title=dict(text="<b>Nice 2012--2018 (1)<b>",
                              font=dict(family='Linux Biolinum O',size=18,color='black')),
                   xaxis=dict(title='Datum'),
                   yaxis=dict(title='DJU', ),
                   paper_bgcolor='rgb(233,233,233)',
                   plot_bgcolor='rgb(233,233,233)',)

fig = go.Figure(data=data, layout=layout)

pyo.iplot(fig)

In [ ]:
df_p9 = df_eCO2.merge(df_DJU, on=['Year','Month'])
df_p9.head(2)

In [ ]:
data=df_p9.copy()
data = data.groupby(by=['Year','Month'],as_index=False).mean()
data = data.sort_values(by=['Year','Month'])
data=data.reset_index(drop=True)
print(len(data))

x=liste_m_y
y1=data['Consommation']
y2=data['dju']
y3=df_p9['Consommation']


trace1 = go.Line(x=x,
                  y=y1,
                  opacity = 0.8,
                  name='Consommation',
                  marker=dict(color='green'))

trace2 = go.Line(x=x,
                  y=y2,
                  opacity = 0.8,
                  name='DJU',
                  marker=dict(color='red'),
                  yaxis='y2')

data = [trace1,trace2]

layout = go.Layout(title=dict(text="<b>Concordance entre la consommation électrique mensuelle et les DJU<b>",
                              font=dict(family='Linux Biolinum O',size=18,color='black')),
                   xaxis=dict(title='Datum'),
                   yaxis=dict(title='KW/h'),
                   yaxis2=dict(
                            title='dju',
                            titlefont=dict(
                                color='rgb(148, 103, 189)'
                            ),
                            tickfont=dict(
                                color='rgb(148, 103, 189)'
                            ),
                            overlaying='y',
                            side='right'),
                   paper_bgcolor='rgb(233,233,233)',
                   plot_bgcolor='rgb(233,233,233)',
                  )


fig = go.Figure(data=data, layout=layout)

pyo.iplot(fig)


# AGGREGATION(week) to make data ligther

In [ ]:
df_p9_agg_month=df_p9.copy().groupby(by=['Year','Month'], as_index=False,).mean()
df_p9_agg_month=df_p9_agg_month.reset_index(drop=True)

df_p9_agg_day=df_p9.copy().groupby(by=['Date'], as_index=False).mean()
df_p9_agg_day['Datetime']=df_p9_agg_day['Date'].astype('datetime64[ns]')
df_p9_agg_day=df_p9_agg_day.set_index('Datetime',drop=True)

df_p9_agg_hour=df_p9[df_p9['Minute']==0].copy()
df_p9_agg_hour=df_p9_agg_hour.reset_index(drop=True)
df_p9_agg_hour['Datetime'] = pd.to_datetime(df_p9_agg_hour['Date'].apply(str)+' '+df_p9_agg_hour['Heures'])
df_p9_agg_hour=df_p9_agg_hour.set_index('Datetime',drop=True)

In [ ]:
print(len(df_p9_agg_month),len(df_p9_agg_day),len(df_p9_agg_hour),len(df_p9))

### <font color='violet'>Heat correction via linear regression<font>

### Linear correction will pbly year deseasonal the consommation no?

In [ ]:
df_p9_agg_month.head(2)

In [ ]:
df_p9_agg_day.head(2)

In [ ]:
df_p9_agg_hour.head(2)

In [ ]:
df_p9_agg_hour['Correction_consommation']=smf.ols(formula='Consommation ~ dju',
                                                 data=df_p9_agg_hour).fit().resid

df_p9_agg_day['Correction_consommation']=smf.ols(formula='Consommation ~ dju',
                                                 data=df_p9_agg_day).fit().resid

df_p9_agg_month['Correction_consommation']=smf.ols(formula='Consommation ~ dju',
                                                 data=df_p9_agg_month).fit().resid

In [ ]:
reg_lin = smf.ols(formula='Consommation ~ dju', data=df_p9_agg_hour).fit()
reg_lin.summary()

## <font color='orange'>SARIMA(p,d,q) (P,D,Q,s)<font>  
    p:AR, AutoRegressiv term making value depends from p past values  
    d:I, Differences  
    q:MA  Movingterm Average extracting residuals MA(original data)
    P:AR (for seasonal component)  
    D:I (for seasonal component)  
    Q:MA (for seasonal component)  
    s:seasonality

In [ ]:
def SARIMAtest(series_train,series_test,p,d,q,sp,sd,sq,s):
    sarima = sm.tsa.statespace.SARIMAX(train,order=(p,d,q),seasonal_order=(sp,sd,sq,s),
                                enforce_stationarity=False, enforce_invertibility=False).fit()
    pred = sarima.predict(train_end,test_end)[1:]
    print('SARIMA model MSE:{}'.format(mean_squared_error(test,pred)))
    pd.DataFrame({'test':test,'pred':pred}).plot();
    plt.show()

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

In [ ]:
from pandas.plotting import autocorrelation_plot

In [ ]:
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(df_p9_agg_month.Correction_consommation, lags=25, alpha=.05, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(df_p9_agg_month.Correction_consommation, lags=25, alpha=.05, ax=ax2)
print("ACF and PACF:")

In [ ]:
data=df_p9_agg_month.copy()
data['Correction_consommation']=data['Correction_consommation']
data=data.reset_index(drop=True)
train_start,train_end = 0,60
test_start,test_end = 60,72
train = data['Correction_consommation'][train_start:train_end].dropna() 
test = data['Correction_consommation'][test_start:test_end].dropna()
SARIMAtest(train,test,1,0,1,1,1,1,12)

In [ ]:
resDiff = sm.tsa.arma_order_select_ic(train,max_ar=2, max_ma=2, ic='aic', trend='nc')
print('ARMA(p,q) =',resDiff['aic_min_order'],'is the best.')

In [ ]:
data=df_p9_agg_day.copy()
data['Correction_consommation']
train_start,train_end = '2012-01-01','2016-12-31'
test_start,test_end = '2017-01-01','2017-12-31'
train = data['Correction_consommation'][train_start:train_end].dropna()
test = data['Correction_consommation'][test_start:test_end].dropna()
SARIMAtest(train,test,1,0,1,1,1,1,365)

In [ ]:
resDiff = sm.tsa.arma_order_select_ic(train, max_ar=2, max_ma=2, ic='aic', trend='nc')
print('ARMA(p,q) =',resDiff['aic_min_order'],'is the best.')

In [ ]:
data=df_p9_agg_hour.copy()
data['Correction_consommation']
train_start,train_end = '2012-01-01 00:00:00','2016-12-31 23:00:00'
test_start,test_end = '2017-01-01 00:00:00','2017-12-31 23:00:00'
train = data['Correction_consommation'][train_start:train_end].dropna()
test = data['Correction_consommation'][test_start:test_end].dropna()
SARIMAtest(train,test,1,0,1,0,0,0,24)

In [ ]:
res = arima.resid
fig,ax = plt.subplots(2,1,figsize=(15,8))
fig = sm.graphics.tsa.plot_acf(res, lags=25, ax=ax[0])
fig = sm.graphics.tsa.plot_pacf(res, lags=25, ax=ax[1])
plt.show()

# <font color='Orange'>Exponential smoothing<font>

 instead of only weighting the time series' last k values, we would weight all available observations while exponentially decreasing the weights as we move further back in time.  
 We can think of α as the smoothing factor or memory decay rate, it defines how quickly we will "forget" the last available true observation. The smaller α is, the more influence the previous observations have and the smoother the series is. In other words, the higher the α

, the faster the method "forgets" about the past.



y^x=α⋅yx+(1−α)⋅y^x−1, we can see that in order to make the prediction for y^x we also need to have the observed value yx.

### <font color='violet'>Simple exponential smoothing<font>  AR(p)I(d)MA(q)    (seasonality, trend, and noise)
    (to weight observations by their age)  
Prediction = Prédiction faite à l’instant précédent corrigée par un terme proportionnel à l’erreur de prévision correspondante.  
#### Effective for stationary data points: Irregular data, No seasonality or trend. The initial value to start the smooth have to be determined (the first of the serie, a mean,...).  
If alpha = 1: naive methode: next = previous  
If alpha = 0: mean methode: next = mean(previous)

In [ ]:
def exponential_smoothingzoom(series, alpha, zoom):
    """
        next value is predicted by the previous one, "fixed" by alpha as : x(p) = x(p-1) + alpha*(ap(-1))
        series - dataset with timestamps
        alpha - float [0.0, 1.0], smoothing parameter proportional with the error made at previous prediction
    """
    end=zoom+1
    # first value is same as series
    result = [series[0]]
    end=zoom+1
    for n in range(1, end):
        result.append(alpha * series[n] + (1 - alpha) * result[n-1])
    return result



def plotExponentialSmoothingzoom(series, alphas, zooms):
    """
        Plots exponential smoothing with different alphas
        
        series - dataset with timestamps
        alphas - list of floats, smoothing parameters
        
    """
    with plt.style.context('seaborn-white'):
        for zoom in zooms:
            end=zoom+1

            plt.figure(figsize=(15, 7))
            for alpha in alphas:
                plt.plot(exponential_smoothingzoom(series, alpha,zoom), label="Alpha {}".format(alpha))
            plt.plot(series[:end].values, 'green', alpha=0.4,label = "Actual")
            plt.legend(loc="best")
            plt.axis('tight')
            plt.title("Exponential Smoothing\nfzoom = {} day(s)".format(zoom))
            plt.grid(True);

In [ ]:
data=df_p9_agg_hour.copy()
plotExponentialSmoothingzoom(data.Correction_consommation, [0.2, 0.9],zooms=freqcy)

### <font color='green'>Of course, alpha=0.9 shapes pretty cloth to actual:  <font>
    -high alpha gives more importance to recent data.
    -series has no trend so SES is efficient.
#### the daily seasonality seems caugth by the SMA(0.9): with a step 1, record(x) ~= record(x+1), with hour frequency. In fact, the daily seasonality is probably not caugth

## <font color='orange'>Double exponential smoothing<font>  
### More efficient for data having trend(s)

if our time series has a trend, we can incorporate that information to do better than just estimating the current level and using that to forecast the future observations. 

In [ ]:
### zoom
def double_exponential_smoothingzoom(series, alpha, beta, zoom):
    """
        series - dataset with timeseries
        alpha - float [0.0, 1.0], smoothing parameter for level
        beta - float [0.0, 1.0], smoothing parameter for trend
    """
        
    end=zoom+1
    # first value is same as series
    result = [series[0]]
    end=zoom+1
    for n in range(1, end):
        if n == 1:
            level, trend = series[0], series[1] - series[0]
        if n >= end: # forecasting
            value = result[-1]
        else:
            value = series[n]
        last_level, level = level, alpha*value + (1-alpha)*(level+trend)
        trend = beta*(level-last_level) + (1-beta)*trend
        result.append(level+trend)
    return result

def plotDoubleExponentialSmoothingzoom(series, alphas, betas, zooms):
    """
        Plots double exponential smoothing with different alphas and betas
        
        series - dataset with timestamps
        alphas - list of floats, smoothing parameters for level
        betas - list of floats, smoothing parameters for trend
    """
    with plt.style.context('seaborn-white'):
        for zoom in zooms:
            end=zoom+1

            plt.figure(figsize=(20, 8))
            for alpha in alphas:
                for beta in betas:
                    plt.plot(double_exponential_smoothingzoom(series, alpha, beta, zoom), label="Alpha {}, beta {}".format(alpha, beta))
            plt.plot(series[:end].values, label = "Actual")
            plt.legend(loc="best")
            plt.axis('tight')
            plt.title("Double Exponential Smoothing\nzoom = {} day(s)".format(zoom))
            plt.grid(True)

In [ ]:
data=df_p9_first_correction.reset_index(drop=True)
plotDoubleExponentialSmoothingzoom(data.Correction_consommation,alphas=[0.9, 0.02], betas=[0.9, 0.02],zooms=[7,30,365])

Although this method can now predictive future values, if we stare closer at the forecast formula y^x+1=ℓx+kTx, we can see that once the trend (T) is estimated to be positive, all future predictions can only go up from the last value in the time series. On the other hand, if the trend (T) is estimated to be negative, all future predictions can only go down. This property makes this method unsuitable for predicting very far out into the future as well. With that in mind, let's now turn towards triple exponential smoothing.

#### <font color='red'>2 factors exponantial smoothing is interessing. Aspecialy with:  <font>
#### <font color='blue'>-high alpha and high beta (blue)<font>  
#### <font color='orange'>-high alpha and low beta (orange)<font>

### <font color='violet'>Tuning double exponential smoothing<font>
#### Alpha and Beta need to be tuned.  
#### *Alpha smooths the serie around the trend,  
#### *Beta smooth the trend itself.  
      
#### The higher they are, the more weight the most recent observations will have,     
#### and the less smoothed the serie extracted from the model will be.  
  
#### This tune will be done automaticaly (by refering to an accuracy rating (loss function) as MSE, SSE, etc...).

## <font color='orange'>Triple exponential smoothing a.k.a. Holt-Winters<font>  
    Main idear: add seasonal component to our model.  
#### Suitable for univariate time series with trend and/or seasonal components. 
#### We saw earlier that series as many seasonabilities.  
    Then the seasonal component of the 3rd point into the season would be exponentially smoothed with the 3rd point of last season, 3rd point two seasons ago, etc.

https://annals-csis.org/proceedings/2012/pliks/118.pdf

In [ ]:
class HoltWinters:
    
    """
    Holt-Winters model with the anomalies detection using Brutlag method
    
    # series - initial time series
    # slen - length of a season -- seasonality
    # alpha, beta, gamma - Holt-Winters model coefficients
    # n_preds - predictions horizon -- refering to the time series periodicity
    # scaling_factor - sets the width of the confidence interval by Brutlag (usually takes values from 2 to 3)
    
    """
    
    
    def __init__(self, series, slen, alpha, beta, gamma, n_preds, scaling_factor=1.96):
        self.series = series
        self.slen = slen#season length
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.n_preds = n_preds
        self.scaling_factor = scaling_factor
        
        
    def initial_trend(self):
        """Common method to initiate the trend: the trend averages across seasons"""
        sum = 0.0
        for i in range(self.slen):
            sum += float(self.series[i+self.slen] - self.series[i]) / self.slen
        return sum / self.slen  
    
    def initial_seasonal_components(self):
        seasonals = {}
        season_averages = []
        n_seasons = int(len(self.series)/self.slen)
        # let's calculate season averages
        for j in range(n_seasons):
            season_averages.append(sum(self.series[self.slen*j:self.slen*j+self.slen])/float(self.slen))
        # let's calculate initial values
        for i in range(self.slen):
            sum_of_vals_over_avg = 0.0
            for j in range(n_seasons):
                sum_of_vals_over_avg += self.series[self.slen*j+i]-season_averages[j]
            seasonals[i] = sum_of_vals_over_avg/n_seasons
        return seasonals   

          
    def triple_exponential_smoothing(self):
        self.result = []#ante prediction values smoothed and predictions
        self.Smooth = []#all values smoothed
        self.Season = []
        self.Trend = []
        self.PredictedDeviation = []
        self.UpperBond = []
        self.LowerBond = []
        
        seasonals = self.initial_seasonal_components()
        
        for i in range(len(self.series)+self.n_preds):
            if i == 0: # components initialization: it can't be predict and first observation of time series is used 
                smooth = self.series[0]
                trend = self.initial_trend()
                self.result.append(self.series[0])
                self.Smooth.append(smooth)
                self.Trend.append(trend)
                self.Season.append(seasonals[i%self.slen])
                
                self.PredictedDeviation.append(0)
                
                self.UpperBond.append(self.result[0] + 
                                      self.scaling_factor * 
                                      self.PredictedDeviation[0])
                
                self.LowerBond.append(self.result[0] - 
                                      self.scaling_factor * 
                                      self.PredictedDeviation[0])
                continue
                
            if i >= len(self.series): # predicting session begins
                m = i - len(self.series) + 1
                self.result.append((smooth + m*trend) + seasonals[i%self.slen])
                
                # when predicting we increase uncertainty on each step
                self.PredictedDeviation.append(self.PredictedDeviation[-1]*1.01) 
                
            else:#calculated deviation for each value in the time serie beween first value of the serie and prediction start
                val = self.series[i]
                last_smooth, smooth = smooth, self.alpha*(val-seasonals[i%self.slen]) + (1-self.alpha)*(smooth+trend)
                trend = self.beta * (smooth-last_smooth) + (1-self.beta)*trend
                seasonals[i%self.slen] = self.gamma*(val-smooth) + (1-self.gamma)*seasonals[i%self.slen]
                self.result.append(smooth+trend+seasonals[i%self.slen])
                
                # Deviation is calculated according to Brutlag algorithm.
                self.PredictedDeviation.append(self.gamma * np.abs(self.series[i] - self.result[i]) 
                                               + (1-self.gamma)*self.PredictedDeviation[-1])
            #confindence boundary          
            self.UpperBond.append(self.result[-1] + 
                                  self.scaling_factor * 
                                  self.PredictedDeviation[-1])

            self.LowerBond.append(self.result[-1] - 
                                  self.scaling_factor * 
                                  self.PredictedDeviation[-1])

            self.Smooth.append(smooth)
            self.Trend.append(trend)
            self.Season.append(seasonals[i%self.slen])

#### <font color='green'>Cross validation to estimate best parameters value<font>
### NEED TO TAKE CARE ABOUT TIME LOGICAL so previous questions are:  
    Length of the duration model will begin to train on?  
    Length of prediction step?  
####    Note that next fold of the cross validation will train on data first train + prediction

In [ ]:

'''TimeSeriesSplit is very interessing for time series:
training fold1 has x values,
trainging fold2 has fold1 values + next x values
.
.
.
testing fold1 has the next x values from traing fold 1
testing fold2 has the next x values from traing fold 2
.
.
.
HOWEVER: n_folds can be limited'''

def timeseriesCVscore(params, series, loss_function=mean_squared_error, slen=365):
    """
        Returns error on CV  
        
        params - vector of parameters for optimization
        series - dataset with timeseries
        slen - season length for Holt-Winters model
    """
     # errors array
    errors = []
    
    values = series.values
    alpha, beta, gamma = params
    
    # set the number of folds for cross-validation
    tscv = TimeSeriesSplit(n_splits=2) 
    
    # iterating over folds, train model on each, forecast and calculate error
    for train, test in tscv.split(values):

        model = HoltWinters(series=values[train], slen=slen, 
                            alpha=alpha, beta=beta, gamma=gamma, n_preds=len(test))
        model.triple_exponential_smoothing()
        
        predictions = model.result[-len(test):]
        actual = values[test]
        error = loss_function(predictions, actual)
        errors.append(error)
        
    return np.mean(np.array(errors))

In [ ]:
#%%time

data=df_p9_first_correction.Correction_consommation.copy()

# initializing model parameters alpha, beta and gamma
x = [0, 0, 0] 

# Minimizing the loss function => optimizing algo
opt = minimize(timeseriesCVscore, x0=x, 
               args=(data, mean_squared_error), 
               method='TNC', bounds = ((0, 1), (0, 1), (0, 1))
              )

# Take optimal values...
alpha_final, beta_final, gamma_final = opt.x
print(alpha_final, beta_final, gamma_final)

# ...and train the model with them, forecasting for the 30 next freq
model = HoltWinters(series=data, slen=365,
                    alpha = alpha_final, 
                    beta = beta_final, 
                    gamma = gamma_final, 
                    n_preds = 365, 
                    scaling_factor = 1)
model.triple_exponential_smoothing()

In [ ]:
def plotHoltWinters(series, slen, plot_intervals=False, plot_anomalies=False):
    """
        series - dataset with timeseries
        plot_intervals - show confidence intervals
        plot_anomalies - show anomalies 
    """
    
    plt.figure(figsize=(20, 10))
    plt.plot(model.result, '-',label = "Model",linewidth=3, color='orange')
    plt.plot(series.values, label = "Data",color='green')
    error = mean_absolute_percentage_error(series.values, model.result[:len(series)])
    plt.title("Mean Absolute Percentage Error: {0:.2f}%".format(error))
    
    if plot_anomalies:
        anomalies = np.array([np.NaN]*len(series))
        anomalies[series.values<model.LowerBond[:len(series)]] = \
            series.values[series.values<model.LowerBond[:len(series)]]
        anomalies[series.values>model.UpperBond[:len(series)]] = \
            series.values[series.values>model.UpperBond[:len(series)]]
        plt.plot(anomalies, "o", markersize=10, label = "Anomalies")
    
    if plot_intervals:
        plt.plot(model.UpperBond, "r--", alpha=.8, label = "Up/Low confidence")
        plt.plot(model.LowerBond, "r--", alpha=.8)
        plt.fill_between(x=range(0,len(model.result)), y1=model.UpperBond, 
                         y2=model.LowerBond, alpha=1, color = "grey")    
        
    plt.vlines(len(series), ymin=min(model.LowerBond), ymax=max(model.UpperBond), linestyles='dashed')
    plt.axvspan(len(series)-12, len(model.result), alpha=0.3, color='lightgrey')
    plt.grid(True)
    plt.axis('tight')
    plt.legend(loc="best", fontsize=13);

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
plotHoltWinters(series=model.series, plot_intervals=False, plot_anomalies=False, slen=365)

In [ ]:
def plotHoltWinters_zoom(series, zoom):
    """
        series - dataset with timeseries
        plot_intervals - show confidence intervals
        plot_anomalies - show anomalies 
    """
    r = randint(a=zoom/2,b=len(series)-zoom/2)
    r1 =int(r-zoom/2)
    r2=int(r+zoom/2)
    x=list(np.arange(0,zoom))
    model.series.index[r1:r2]
    res=model.result[r1:r2]
    val=list(model.series.values[r1:r2])
    start=model.series.index[r1]
    end=model.series.index[2]
    
    plt.figure(figsize=(20, 10))
    plt.plot(res, label = "Model")
    plt.plot(val, label = "Data")
    error = mean_absolute_percentage_error(np.array(val), res)
    plt.title("Mean Absolute Percentage Error: {:.2f}%\nRandom periode : {} to {}".format(error,start,end))
    plt.legend(loc="best", fontsize=13);

In [ ]:
plt.figure(figsize=(25, 5))
plt.plot(model.PredictedDeviation)
plt.grid(True)
plt.axis('tight')
plt.title("Brutlag's predicted deviation");

In [ ]:
def linear_weight_moving_average(signal, period):
    buffer = [np.nan] * period
    for i in range(period, len(signal)):
        buffer.append(
            (signal[i - period : i] * (np.arange(period) + 1)).sum()
            / (np.arange(period) + 1).sum()
        )
    return buffer

data=df_p9_first_correction.copy()

print(len(data))

x=data['Datetime']
y1=data['Correction_consommation']
y2=linear_weight_moving_average(y1,7)
y4=linear_weight_moving_average(y1,365)

trace1 = go.Line(x=x,
                  y=y1,
                  opacity = 0.8,
                  name='Consommation corrigée',
                  marker=dict(color='red'))

trace2 = go.Line(x=x,
                  y=y2,
                  opacity = 0.8,
                  name='LMA(7)',
                  marker=dict(color='green'))

trace3 = go.Line(x=x,
                  y=y4,
                  opacity = 0.8,
                  name='LMA(365)',
                  marker=dict(color='blue'))


data = [trace1,trace2,trace3]

layout = go.Layout(title=dict(text="<b><b>",
                              font=dict(family='Linux Biolinum O',size=18,color='black')),
                   yaxis=dict(title='KW/h'),
                   xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=7,
                     label='7d',
                     step='day',
                     stepmode='backward'),
                dict(count=1,
                     label='1y',
                     step='year',
                     stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(
            visible = True
        ),
        type='date'
    )
)


fig = go.Figure(data=data, layout=layout)

pyo.iplot(fig)

